# Generador de resumenes

Este cuaderno tiene como próposito generar un resumen de un texto de términos y condiciones utilizando el modelo más óptimo discutido en este proyecto. Para esto, necesitamos de un texto en formato txt, el cual será modificado utilizando la variable nombre_texto para el nombre del texto, con el sufijo .txt . Este texto deberá estar en la misma ubicación que este cuaderno. 

In [5]:
#Importar elementos necesarios de las librerías
import os, shutil, re, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sys import setrecursionlimit
from rouge import Rouge
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SpanishStemmer
from scipy.stats import binom
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix

#Aumentar la el limite de la recursión
setrecursionlimit(10**5)

In [6]:
ruta_texto = os.getcwd()
nombre_texto = "waze-demo.txt"

In [7]:
#Funciones auxiliares

def tokenize_and_stemm(path,file_name):
    #Tokeniza un texto y aplica el stemming de snowball
    file = []
    stemmer = SpanishStemmer()
    text = PlaintextCorpusReader(path, file_name) #Cambiar ruta por '/'
    tokens = text.words()
    tokens = [stemmer.stem(token) for token in tokens]
    text = ' '.join(tokens)
    return text

def tokenize_sentence(path, file_name):
    #Retorna un documento tokenizado por frases
    doc = []
    text = PlaintextCorpusReader(path, file_name)
    paragraphs = text.paras()
    for paragraph in paragraphs:
        for sentence in paragraph:
            low, i = 0,0
            while i < len(sentence):
                token = sentence[i].split('.')
                if len(token)-1:
                    doc.append(sentence[low:i])
                    low=i+1
                    i+=2
                else:
                    i+=1
            if low!=i-1:
                doc.append(sentence[low:i])
    return doc

def preprocess(doc, stopwords, stemmer):
    #Aplica el preprocesamiento establecido
    #Adicionalmente, retorna el documento original sin las filas vacías por el preprocesamiento 
    doc_preprocesed, doc_reduced = [], []
    for original_sentence in doc:
        preprocessed_sentence = []
        for token in original_sentence:
            if stemmer.stem(token) not in stopwords:
                preprocessed_sentence.append(stemmer.stem(token))
        if len(preprocessed_sentence) and preprocessed_sentence not in doc_preprocesed:
            doc_preprocesed.append(preprocessed_sentence)
            doc_reduced.append(original_sentence)
    return doc_preprocesed, doc_reduced

def get_dictionaries(doc):
    #Retorna un par de diccionarios que relacionan una frase con un id, y un id con una frase.
    sentence2id, id2sentence = {},{}
    n_sentences = len(doc)
    for i in range(n_sentences):
        sentence = ' '.join(doc[i])
        if sentence not in sentence2id:
            sentence2id[sentence] = i
            id2sentence[i] = sentence
    return sentence2id, id2sentence

def reverse_dict(dictionary):
    #Invierte el sentido de un diccionario
    reverse = {}
    for k,v in dictionary.items():
        reverse[v] = k
    return reverse

def log_likelihood_ratio(bag_of_words):
    #Calcula la matriz -2 lambda
    tokens_per_document = bag_of_words.sum(1)
    ocurrences = bag_of_words.sum(0)
    total_tokens = bag_of_words.sum(0).sum(1)[0,0]
    n_documents,m_tokens = bag_of_words.shape
    loglikelihood_general = []
    for i in range(m_tokens):
        ocurrences_general = ocurrences[0,i]
        loglikelihood = binom.pmf(ocurrences_general, total_tokens,ocurrences_general/total_tokens)
        loglikelihood_general.append(loglikelihood)
    data = []
    for i in range(n_documents):
        row = []
        tokens_input = tokens_per_document[i,0]
        tokens_background = total_tokens - tokens_input
        for j in range(m_tokens):
            ocurrences_total = ocurrences[0,j]
            ocurrences_input = bag_of_words[i,j]
            ocurrences_background = ocurrences_total - ocurrences_input
            loglikelihood_input = binom.pmf(ocurrences_input, tokens_input, ocurrences_input/tokens_input)
            loglikelihood_background = binom.pmf(ocurrences_background, tokens_background, ocurrences_background/tokens_background)
            ratio = loglikelihood_general[j]/(loglikelihood_input*loglikelihood_background)
            row.append(ratio)
        data.append(row)
    lambda_matrix = np.array(data,dtype=np.float64)
    return lambda_matrix

def corpus_topic_signatures(log_likelihood_matrix, treshold):
    #Crea una matriz con los topic signatures a partir de la matriz -2lambda y el umbral
    data, col_index, row_index = [],[],[]
    n_documents, m_tokens = log_likelihood_matrix.shape
    for i in range(n_documents):
        for j in range(m_tokens):
            if log_likelihood_matrix[i,j] >= treshold:
                row_index.append(i)
                col_index.append(j)
                data.append(1)
    data = np.array(data)
    row_index = np.array(row_index)
    col_index = np.array(col_index)
    topic_signatures = csr_matrix((data,(row_index,col_index)),shape=(n_documents, m_tokens),dtype=np.float64)
    return topic_signatures

def document_topic_signatures(document, topic_signatures, token2id, doc_id):
    #Construye una matriz de un solo documento de acuerdo a la matriz de topic signatures 
    data,row_index,col_index = [],[],[]
    topic_signatures = topic_signatures.toarray()
    n_sentences, m_tokens = len(document), len(token2id)
    for i in range(n_sentences):
        sentence = document[i]
        j = 0
        for token in sentence:
            if token in token2id and topic_signatures[doc_id,token2id[token]] == 1:
                if (j==0) or (j>0 and token2id[token] not in col_index[-j:]):
                    topic_signature_value = topic_signatures[doc_id,token2id[token]]
                    data.append(topic_signature_value)
                    row_index.append(i)
                    col_index.append(token2id[token])
                    j+=1
    data = np.array(data)
    row_index = np.array(row_index)
    col_index = np.array(col_index)
    topic_signatures_document = csr_matrix((data,(row_index,col_index)),shape=(n_sentences, m_tokens),dtype=np.float64)
    return topic_signatures_document

def generate_summary(topic_signatures, n_sentences, sentence2id):
    #Genera un resumen con un total de n_sentences frases
    total_sentences = topic_signatures.shape[0]
    assert n_sentences < total_sentences
    topic_signatures = topic_signatures.toarray().sum(1)
    df = pd.DataFrame(topic_signatures,columns = ['topic signatures'])
    df['sentence'] = sentence2id.keys()
    df = df.sort_values(by='topic signatures', ascending = False)
    summary = []
    i,flag=0,1
    while i < n_sentences and flag:
        sentence = df.iloc[i]['sentence']
        flag = df.iloc[i]['topic signatures']
        if flag:
            summary.append(sentence)
        i+=1
    assert summary != []
    return summary

In [8]:
#Cargar las stopwords obtenido en preprocesamiento.
filename = 'stopwords.pkl'
stopwords = pickle.load(open(filename, 'rb'))

#Carga el corpus usado para el preprocesamiento
filename = 'corpus_single_string.pkl'
corpus_single_string = pickle.load(open(filename, 'rb'))

#Tokenización y stemming en un solo string
texto = tokenize_and_stemm(ruta_texto, nombre_texto)

#if texto not in corpus_single_string: 
corpus_single_string.append(texto)
    
#Tokenización por frases para los resumenes
texto_frases = tokenize_sentence(ruta_texto, nombre_texto)
stemmer = SpanishStemmer()
texto_frases_con_stemm, texto_frases = preprocess(texto_frases, stopwords, stemmer)

#Diccionarios
frase2id, id2frase = get_dictionaries(texto_frases)

#Matriz de frecuencia o bag of words
vectorizer = CountVectorizer(stop_words = stopwords)

bag_of_words = vectorizer.fit_transform(corpus_single_string)

#Diccionario que relaciona tokens con un identificador único
token2id = vectorizer.vocabulary_

#Diccionario que un identificador con un token
id2token = reverse_dict(token2id)

#Matriz de verosimilitud logarítmica (lambda)
lambda_matrix = log_likelihood_ratio(bag_of_words)

#Detectar topic signatures de todo el corpus
umbral = 5.0239
topic_signatures = corpus_topic_signatures(lambda_matrix,umbral)

#Aplicar topic signatures al texto
indice_texto = -1
topic_signatures = document_topic_signatures(texto_frases_con_stemm, topic_signatures, token2id, indice_texto)

#Generar el resumen
n_frases = 10
resumen = generate_summary(topic_signatures, n_frases, frase2id)
resumen = '. '.join(resumen)
resumen

'Waze puede recoger y registrar con qué frecuencia y durante cuánto tiempo utilizas la Aplicación ; qué tipo de dispositivo tienes ; tus identificadores únicos ; el tipo y la versión del sistema operativo ; el uso de la batería ; las páginas web o aplicaciones de terceros que visitas o utilizas , o con las que interactúas a través de la Aplicación ; la información que has consultado en nuestros Servicios ; los anuncios que te muestra , los que ves o en los que haces clic ; el hecho de que hayas usado los Servicios para comunicarte con otros usuarios y el hecho de que hayas usado servicios o aplicaciones de terceros a través de Waze para comunicarte con terceros , y la duración de dichas comunicaciones ; la dirección de protocolo de Internet ( IP ) y el nombre del dominio con el que accedes a los Servicios ; y la ubicación geográfica del dispositivo que usas para iniciar sesión en Waze y utilizar la Aplicación o el Sitio Web. Waze vinculará toda tu información con tu Cuenta y con ese no